In [1]:
import h5py as h5
#from h5glance import H5Glance
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import gvar as gv
import pandas as pd
import os 
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib notebook
#import h5ls
import lsqfit
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import yaml
from pathlib import Path
import importlib

In [2]:
#import local files
#sys.path.append("../")
import gmo_fitter as gmo 
import i_o 
import gmo_fit_analysis as fa
import load_data_priors as ld
import mass_relations as ma
matplotlib.rcParams['figure.figsize'] = [10, 8]

In [3]:
from input_files import a12m180L
# from input_files import a12m180S
# from input_files import a12m220
# from input_files import a12m220L
# from input_files import a12m220S
# from input_files import a12m220XL
# from input_files import a12m220_ms
# from input_files import a12m260
# from input_files import a12m310
# from input_files import a12m310L
# from input_files import a15m130
# from input_files import a15m310


p_dict = a12m180L.p_dict
abbr = p_dict['abbr']

In [4]:
import platform
if platform.system() == 'Darwin':
    file = '/Users/grantdb/lqcd/data/c51_2pt_octet_decuplet.h5'
else:
    file = '/home/gmoney/lqcd/data/c51_2pt_octet_decuplet.h5'
with h5.File(file,"r") as f:
    ensembles = {}
    ensembles = list(f.keys())
    # print(ensembles)

In [6]:
# hacky way to initialize all raw corr data 
importlib.reload(ld)
nucleon_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='proton')
lam_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='lambda_z')
xi_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='xi_z')
sigma_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='sigma_p')
piplus_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='piplus')
kplus_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='kplus')
delta_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='delta_pp')
gmo_ratio_raw = ld.G_gmo(file,p_dict['abbr'],log=False)
# eff_mass = ma.gmo_eff_mass(gmo_out=gmo_ratio_raw,dt=1)
# gmo_ratio_raw


/home/gmoney/lqcd/gmo_relation/load_data_priors.py:55: RuntimeWarning: invalid value encountered in power
  * np.power(temp[('proton', smr)], -2/3)


In [7]:
prior_nucl = {}
prior = {}
states=p_dict['gmo_states_all']
newlist = [x for x in states]
for x in newlist:
    path = os.path.normpath("./priors/{0}/{1}/prior_nucl.csv".format(p_dict['abbr'],x))
    df = pd.read_csv(path, index_col=0).to_dict()
    for key in list(df.keys()):
        length = int(np.sqrt(len(list(df[key].values()))))
        prior_nucl[key] = list(df[key].values())[:length]
        # prior_nucl['gmo_E'] = list([np.repeat(gv.gvar('0.0030(27)'),8)])
    prior = gv.gvar(prior_nucl)
# prior_gmo = {}
# prior['gmo_E'] = gv.gvar('0.0030(27)') #this shouldnt be priored like this? #

In [8]:
importlib.reload(gmo)
importlib.reload(fa)
importlib.reload(a12m180L)
importlib.reload(ma)
model_type = 'simult_baryons'
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior,
nucleon_corr_data=nucleon_corr,lam_corr_data=lam_corr, xi_corr_data=xi_corr,sigma_corr_data=sigma_corr,delta_corr_data=None,gmo_corr_data=None,
piplus_corr_data=None,kplus_corr_data=None,model_type=model_type)
# print(gmo_.get_fit().format(maxline=True))
print(gmo_)
# print(gmo_.get_m_4(t=None))
fit_out = gmo_.get_fit()
gmo_.plot_effective_mass( t_plot_min=0,t_plot_max=15, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)
out_path = 'fit_results/{0}/'.format(p_dict['abbr'],model_type)

if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
ld.pickle_out(fit_out=fit_out, out_path=out_path,species='all')
corr_fits = {} # resulting fit params, need E0 for gmo study 
mass = {}
gmo_states = ['sigma','xi','proton','lam']
abbr =p_dict['abbr']

# for ens in ensembles:
mass[p_dict['abbr']] = {}
out_path = 'fit_results/{0}/'.format(p_dict['abbr'],model_type)
for corr in gmo_states:
    corr_fits = gv.load(out_path+'fit_params_all')
    mass[p_dict['abbr']][corr] = corr_fits['p'][corr+'_E0']

Model Type:simult_baryons
	 N_{corr} = 2	
	 t_{corr} = [4, 15]
Fit results: 
	 GMO rln = 0.0041(37)
	 M_4 posterior =    -0.1609(77)
	 Centroid posterior =    0.7032(14)
	 lam_E0=  0.6738(19)
	 1st e.s:  0.394(35)
	 proton_E0=  0.5905(46)
	 1st e.s:  0.350(56)
	 xi_E0=  0.78184(96)
	 1st e.s:   0.449(18)
	 sigma_E0=  0.7357(19)
	 1st e.s:  0.448(28)
Least Square Fit:
  chi2/dof [dof] = 1 [72]    Q = 0.44    logGBF = 1747.7

Parameters:
        gmo_E0 0           0.3 (2.5)        [    0.3 (2.5) ]  
          z_PS 0    2e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
               1   -4e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
          z_SS 0        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
               1        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
    sigma_z_PS 0       3.105(64)e-06        [ 0.0(3.3)e-06 ]  
               1        4.86(48)e-06        [ 0.0(3.3)e-06 ]  *
    sigma_z_SS 0        8.27(16)e-07        [ 4.4(4.4)e-07 ]  
               1        6.69(57)e-07        [ 4.4(4.4)e

/home/gmoney/lqcd/gmo_relation/gmo_fit_analysis.py:430: RuntimeWarning: invalid value encountered in log
  return {key : 1/dt * np.log(nucleon_corr_gv[key] / np.roll(nucleon_corr_gv[key], -1))


<IPython.core.display.Javascript object>

FileNotFoundError: [Errno 2] No such file or directory: 'fit_results/a12m180L/fit_params_all'

In [9]:
importlib.reload(gmo)
importlib.reload(fa)
importlib.reload(a12m180L)
importlib.reload(ma)
model_type = 'simult_baryons_gmo'
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior,
nucleon_corr_data=nucleon_corr,lam_corr_data=lam_corr, xi_corr_data=xi_corr,sigma_corr_data=sigma_corr,delta_corr_data=None,gmo_corr_data=gmo_ratio_raw,
piplus_corr_data=None,kplus_corr_data=None,model_type=model_type)
print(gmo_)
gmo_eff_mass = gmo_.get_gmo_effective(gmo_ratio=gmo_ratio_raw)
print(gmo_eff_mass)
gmo_.plot_gmo_effective_mass(effective_mass=gmo_eff_mass,t_plot_min=0,t_plot_max=20, model_type=model_type,fig_name='plots/{0}_{1}_eff'.format(abbr,model_type),show_fit=True)
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])

if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
ld.pickle_out(fit_out=fit_out, out_path=out_path,species='all')
corr_fits = {} # resulting fit params, need E0 for gmo study 
mass = {}
gmo_states = ['sigma','xi','proton','lam']
abbr =p_dict['abbr']

# for ens in ensembles:
mass[p_dict['abbr']] = {}
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
for corr in gmo_states:
    corr_fits = gv.load(out_path+'fit_params_all')
    mass[p_dict['abbr']][corr] = corr_fits['p'][corr+'_E0']
        
# print(mass)
gmo_raw =  ma.GMO(mass_dict=mass,file=file,abbr=abbr,baryons=True)
gmo_rln = gmo_raw.gmo_rln()
m_4 = gmo_raw.m_4()
centroid = gmo_raw.centroid()
delta_gmo = gmo_raw.gmo_violation()

Model Type:simult_baryons_gmo
	 N_{corr} = 2	
	 t_{corr} = [4, 15]
Fit results: 
	 GMO rln = 0.0049(15)
	 M_4 posterior =    -0.1616(69)
	 Centroid posterior =    0.7029(13)
	 lam_E0=  0.6739(16)
	 1st e.s:  0.399(28)
	 proton_E0=  0.5894(34)
	 1st e.s:  0.339(40)
	 xi_E0=  0.78188(93)
	 1st e.s:   0.449(18)
	 sigma_E0=  0.7356(19)
	 1st e.s:  0.447(27)
Least Square Fit:
  chi2/dof [dof] = 0.98 [90]    Q = 0.52    logGBF = 1854.5

Parameters:
        gmo_E0 0           0.3 (2.5)        [    0.3 (2.5) ]  
          z_PS 0    1e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
               1   -5e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
          z_SS 0        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
               1        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
    sigma_z_PS 0       3.102(63)e-06        [ 0.0(3.3)e-06 ]  
               1        4.86(48)e-06        [ 0.0(3.3)e-06 ]  *
    sigma_z_SS 0        8.26(15)e-07        [ 4.4(4.4)e-07 ]  
               1        6.70(56)e-07        [ 4.

/home/gmoney/lqcd/gmo_relation/gmo_fit_analysis.py:268: RuntimeWarning: invalid value encountered in log
  return {key : 1/dt * np.log(gmo_ratio[key] / np.roll(gmo_ratio[key], -1))


<IPython.core.display.Javascript object>

FileNotFoundError: [Errno 2] No such file or directory: 'fit_results/a12m180L/fit_params_all'

**

In [ ]:
if p_dict['make_plots']:
    plots = gmo_.make_plots()
    output_dir = 'fit_results/{0}/{1}_{0}.pdf'.format(p_dict['abbr'],model_type)
    if os.path.exists(output_dir):
        pass
    else:
        os.mkdir(output_dir)
    output_pdf = PdfPages(output_dir)
    for plot in plots:
        if plot is not None:
            output_pdf.savefig(plot)
    if p_dict['show_many_states']:
        output_pdf.savefig(fit_ensemble.plot_stability(model_type='corr', n_states_array=[1, 2, 3, 4]))
    output_pdf.close()

<IPython.core.display.Javascript object>

((7, 15, 2),)
0 PS


/home/gmoney/lqcd/gmo_mq/fit_manager.py:122: RuntimeWarning: invalid value encountered in log
  return {key : 1/dt * np.log(nucleon_corr_gv[key] / np.roll(nucleon_corr_gv[key], -1))


<IPython.core.display.Javascript object>

1 SS
((7, 15, 2),)


...

: 

: 

: 

: 

: 

## bootstrapping ##

bs resampling and redo fits 

keep track of seed 

gauge average bc approximate integral that way -> physical point

data from configs, resample gauge configs, average them to get result of integral

rand sample m times (gauge field configs) to compute bs sample 

array n x m, average over m (n number of resamples)

each resample n correspond to an integral to be computed by avg over m configurations

https://lsqfit.readthedocs.io/en/latest/lsqfit.html
1) make a large number of “bootstrap copies” of the original input data and prior that differ from each other by random amounts characteristic of the underlying randomness in the original data

2) repeat the entire fit analysis for each bootstrap copy of the data, extracting fit results from each 

3) use the variation of the fit results from bootstrap copy to bootstrap copy to determine an approximate probability distribution (possibly non-gaussian) for the fit parameters and/or functions of them: the results from each bootstrap fit are samples from that distribution.

In [ ]:
def bs_to_gvar(data, bs_list):
    bs_M = data['smr'].shape[0] # bs_M would be the same if you used 'ps' instead
    #bs_list = np.random.randint(low=0, high=bs_M, size=(bs_N, bs_M))
    
    temp_dict = {}
    for key in data.keys():
        temp = data[key][bs_list[0:]
        temp_dict[key] = np.mean(temp, axis=0).real
        
    for k in range(1, bs_N):
        for key in data.keys():
            temp = data[key][bs_list[k:]
            temp_dict[key] = np.vstack((temp_dict[key], np.mean(temp, axis=0).real))
    
    output = {}
    #return temp_dict
    for key in data.keys():
        mean = np.mean(data[key], axis=0)
        unc = np.cov(temp_dict[key], rowvar=False)

        
        output[key] = gv.gvar(mean,unc)
    return output

nucleon_bs_list = bs_to_gvar(nucleon_corr_data, 500)
axial_bs_list   = bs_to_gvar(axial_fh_num_data,500)
vector_bs_list  = bs_to_gvar(vector_fh_num_data,500)
#nucleon_bs_list


SyntaxError: invalid syntax. Perhaps you forgot a comma? (536750190.py, line 7)

In [ ]:
# Make bs_N fits, one for each of the resampled correlators
fit_keys = fit_out.p.keys()
output = {key : [] for key in fit_keys}

for j, bsfit in enumerate(fit_out.bootstrapped_fit_iter(datalist=bs_correlators_gv)):
    for key in fit_keys:
        # Save the best estimate for the central value 
        # of each parameter of each fit
        p = bsfit.pmean[key]
        output[key].append(p)

# # print results -- should be similar to previous results
table = gv.dataset.avg_data(output, bstrap=True)
print(gv.tabulate(table))

NameError: name 'bs_correlators_gv' is not defined